### Import

In [17]:
import torch
from Dataset.bank_dataset import BankTxnDataset, pad_collate_fn
from Models.transformer import TransformerClassifier
from Config.config import load_config
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import tqdm

### Loading Config

In [18]:
cfg = load_config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device: \033[92m{device}\033[0m")

TypeError: Config.__init__() got an unexpected keyword argument 'model'

### Loading Dataset

In [10]:
train_ds = BankTxnDataset(cfg, split="train")
print(f"Total number of training data: \033[92m{len(train_ds.data)}\033[0m")

train_loader = DataLoader(
	train_ds,
	batch_size=cfg.parameter['batchSize'],
	shuffle=True,
	num_workers=4,
	pin_memory=True,                # speeds host→GPU copies
	collate_fn=pad_collate_fn
)

AttributeError: 'Config' object has no attribute 'model'

In [4]:
sample_batch = next(iter(train_loader))
x_sample = sample_batch[0]
print(f"Input tensor shape: {x_sample.shape}")
actual_feat_dim = x_sample.shape[2]
print(f"Feature dimension from data: {actual_feat_dim}")

KeyboardInterrupt: 

### Loading Model & Optimizer

In [ ]:
model = TransformerClassifier(
	feat_dim=actual_feat_dim,
	d_model=cfg.parameter['d_model'],
	nhead=cfg.parameter['attention_head'],
	num_layers=cfg.parameter['num_layers'],
	num_classes=1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=cfg.parameter['learningRate'])
scaler    = torch.cuda.amp.GradScaler()  # optional mixed‑precision

c:\Users\Sean\anaconda3\envs\BankWatchList\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\Sean\AppData\Local\Temp\ipykernel_12032\41187433.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler    = torch.cuda.amp.GradScaler()  # optional mixed‑precision


### Training

In [ ]:
epochs = cfg.parameter['epochs']
print(f"Starting training for {epochs} epochs on {device}")
model.train()
# Create list to store average loss per epoch
avg_losses = []

for epoch in range(1, epochs+1):
        epoch_loss = 0.0
        pbar = tqdm.tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", ncols=80)
        for x, lengths, y in pbar:
            x, lengths, y = x.to(device), lengths.to(device), y.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                logits = model(
                    x,
                    src_key_padding_mask=(torch.arange(x.size(1), device=device)
                                          .unsqueeze(0)
                                          .ge(lengths.unsqueeze(1)))
                )
                loss = torch.nn.functional.binary_cross_entropy_with_logits(
                    logits.squeeze(), y
                )
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            batch_loss = loss.item()
            epoch_loss += batch_loss
            # update tqdm bar
            pbar.set_postfix(loss=f"{batch_loss:.4f}")

        avg_loss = epoch_loss / len(train_loader)
        avg_losses.append(avg_loss)  # Store the average loss
        print(f"Epoch {epoch} completed — avg loss: {avg_loss:.4f}")

# After training, plot the loss curve
plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs+1), avg_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(1, epochs+1))

# Add loss values as text annotations on the plot
for i, loss in enumerate(avg_losses):
    plt.text(i+1, loss, f'{loss:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

Starting training for 256 epochs on cpu


Epoch 1/256:   0%|                                      | 0/781 [00:00<?, ?it/s]C:\Users\Sean\AppData\Local\Temp\ipykernel_12032\2459476516.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/256:  17%|██▌            | 131/781 [00:44<03:41,  2.93it/s, loss=0.0326]


KeyboardInterrupt: 